* device_id: l'identificatore univoco


* iaq (Indoor Air Quality): misura o un indice che valuta la qualità dell'aria interna. Calcolato considerando diversi parametri come la concentrazione di particelle in sospensione, la presenza di composti chimici nocivi o altre metriche legate alla qualità dell'aria.
* * trovare il valore appropriato e chiedere se è inversamente proporzionale o meno



* electrosmog_lf: riferimento alla misura o alla quantità di elettrosmog a bassa frequenza nell'ambiente. L'elettrosmog si riferisce alle radiazioni elettromagnetiche prodotte da dispositivi elettronici o reti wireless. La misura potrebbe indicare la quantità di radiazioni elettromagnetiche presenti nell'ambiente.


* wifi_level: il livello di segnale o la potenza del segnale WiFi nell'ambiente. In dBm (decibel-milliwatt) o in un'altra scala di misura. Questa informazione può essere utile per valutare la copertura o la qualità del segnale WiFi nelle diverse aree dell'ufficio o dell'edificio.


* wifi_n: numero di reti WiFi rilevate o disponibili nell'ambiente. Indica la quantità di reti wireless presenti nelle vicinanze del dispositivo di rilevamento.


* temperature: temperatura ambientale nell'area monitorata.


* humidity: l'umidità relativa dell'aria nell'ambiente. L'umidità relativa si riferisce alla quantità di vapore acqueo presente nell'aria rispetto alla massima quantità che l'aria potrebbe contenere a una determinata temperatura.


* air_pressure: pressione atmosferica nell'ambiente. La pressione atmosferica è la forza esercitata dall'atmosfera sulla superficie terrestre e viene misurata solitamente in unità come Pascal (Pa) o millibar (mbar).


* ambient_light: livello di luce ambientale nell'ambiente. Potrebbe essere espresso in unità di illuminazione come lux o in una scala relativa.


* tvoc (Total Volatile Organic Compounds): indicazione della qualità dell'aria in relazione alla presenza di composti organici volatili. I composti organici volatili sono sostanze chimiche che possono essere rilasciate nell'aria da materiali o attività presenti nell'ambiente. Alcuni esempi di composti organici volatili includono formaldeide, benzene, toluene e altri idrocarburi. Un livello elevato di TVOC potrebbe indicare una possibile presenza di inquinanti nell'aria, che potrebbero influire sulla qualità dell'ambiente interno.



Qualità dell' aria
VOC	ppb
CO2	ppm
CO2 e	ppm
PM 10	µg/m3
PM 2.5	µg/m3
IAQ	N.A.

Comfort Ambientale
Temp	°C
Pressione	mbar
Lux	lux
Umidità	RH%
Suono	dB

Elettrosmog
Elett. HF	V/m
Elett. LF	nT
Wifi Liv.	dBm
Wifi N.	N.A.


Normalizzazione delle variabili:

Normalizza la temperatura (T) nell'intervallo da 0 a 1: T_norm = (T - T_min) / (T_max - T_min)
Normalizza l'umidità (H) nell'intervallo da 0 a 1: H_norm = (H - H_min) / (H_max - H_min)
Calcolo del punteggio dell'IAQ:

Assegna un peso alla temperatura (w_T) e all'umidità (w_H).
Calcola il punteggio dell'IAQ (IAQ_score) come la media ponderata dei valori normalizzati: IAQ_score = w_T * T_norm + w_H * H_norm

--------------------------------------------------------------------------
capire se c'è correlazione dei dati interni e dati esterni con stessi periodi etc...correlazione diretta o indiretta

predittiva--> cosa succede nei prossimi mesi...
    what if analysis se temp sale 1 grado cosa accade aglei altri parametri, correlazione tra i vari parametri
    anomaly detection, dati 2 3 anni di dati capire se ci sono eventi che vanno fuori il comportamento attes--->costruire modello comportamentale "standard" e trovari i picchi...


In [2]:
from parsing_table import ParsingTable
from datetime import date

import pandas as pd
import numpy as np
import os

#cap133 fuori
#cap153 centro
################# OLD
# path = './resources/'
# save_path = './resources/clean_file/'
# if not os.path.exists(save_path):
#     os.mkdir(save_path)
# dir_list = os.listdir(path)
# pool = {}
# for file in dir_list:
#     if file.endswith('.csv'):
#         temp:pd.DataFrame
#         naming = file.split('.')[0].split('_')[1].replace('0', '') + '_' + file.split('.')[0].split('_')[0]
#         print(naming)
#         temp = pd.read_csv(path + file, engine='python')
#         # first dataset cleaning
#         temp = temp[temp['electrosmog'].astype(str).str.find('-') == -1]
#         pool[naming] = temp
#         temp.to_csv(save_path+naming+'.csv',index=False)
#
# date_format ='%Y-%m-%d %H:%M:%S'
################

input_path = './resources/clean_file/'
dir_list = os.listdir(input_path)
pool = {}
pt= ParsingTable()
for file in dir_list:
    if file.endswith('.csv'):
        temp: pd.DataFrame
        naming = file.split('.')[0]
        print(naming)
        temp = pt.parse(input_path=input_path+file)
        pool[naming] = temp



CAP133_1


C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc[res.notna()] = pd.to_numeric(res.loc[res.notna()], errors='coerce')
C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc[res.notna()] = res.loc[res.notna()].apply(ParsingTable.date_converter)
C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:92: DtypeWarning: Columns (13,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(path),
C:\Users\lucab\Documents\Python\Ass

CAP133_2


C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc[res.notna()] = res.loc[res.notna()].apply(ParsingTable.date_converter)
C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc[res.notna()] = pd.to_numeric(res.loc[res.notna()], errors='coerce')


CAP133_3


...

C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:92: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(path),
C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc[res.notna()] = pd.to_numeric(res.loc[res.notna()], errors='coerce')
C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc[res.notna()] = pd.to_numeric(res.loc[res.notna()], errors='coerce')
C:\Users\lucab\Do

CAP153_2


C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc[res.notna()] = res.loc[res.notna()].apply(ParsingTable.date_converter)


CAP153_3


C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:92: DtypeWarning: Columns (13,14,15,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(path),
C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc[res.notna()] = pd.to_numeric(res.loc[res.notna()], errors='coerce')
C:\Users\lucab\Documents\Python\Assignement_RBS\parsing_table.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc[res.notna()] = pd.to_numeric(res.loc[res.notna()], errors='coerce')
C:\Users\lucab\Documents\Python\

In [15]:
type( pool['CAP153_3']['rom_unixtime(ts)'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [24]:
only_date= pool['CAP153_3'][['iaq','rom_unixtime(ts)']]
print(only_date.drop_duplicates(['iaq']))

          iaq     rom_unixtime(ts)
0        25.0  2020-01-15 11:32:41
6        25.3  2020-01-15 11:58:56
11       28.4  2020-01-15 12:23:49
15       27.7  2020-01-15 12:43:43
18       32.4  2020-01-15 12:58:37
...       ...                  ...
156081  279.1  2021-11-30 21:21:41
156137  271.0  2021-12-01 01:40:01
156138  310.3  2021-12-01 01:44:58
158834  261.5  2021-12-10 08:54:50
170454  269.6                  NaN

[2334 rows x 2 columns]


In [ ]:
CAP133_1 = pool['CAP133_1']
test_dates = CAP133_1['from_unixtime(ts)']

In [ ]:
test_dates

In [ ]:
#trovare iaq in abase alla fascia temporale
caap = 'CAP153_3'
device_number = \
    pool[caap].assign(iaq_type=pool[caap]['iaq'], iaq_n=pool[caap].groupby('iaq')['iaq'].transform('count'))[
        ['iaq_type', 'iaq_n']].drop_duplicates()
device_number

In [ ]:
pool_33_3.loc[pool_33_3['main_power'] != 100, 'main_power']

In [ ]:
device_number: pd.DataFrame = pd.DataFrame()
device_number = device_number.assign(device_id=pool_33_3['device_id'],
                                     device_n=pool_33_3.groupby('device_id')['device_id'].transform(
                                         'count')).drop_duplicates()
# pool_33_3['dev_n'] = pool_33_3.groupby('device_id')['device_id'].transform('count')
device_number

In [ ]:
# oaq --> iaq (con stesse variabili conosciute) ==> troviamo efficienza filtraggio interno